# Bootstrapping Exploration

Based on the PhD. Dissertation,

https://www.proquest.com/docview/2529344852?pq-origsite=gscholar&fromopenview=true

In [60]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

## Information about the device

In [53]:
class Quantity:
    def __init__(self, value: float, min_value: float, max_value: float, unit: str) -> None:

        base_units = ["V", "A"]
        if unit not in base_units:
            prefixes = {
                'y': -24,
                'z': -21,
                'a': -18,
                'f': -15,
                'p': -12,
                'n': -9,
                'u': -6,
                'm': -3,
                'c': -2,
                'd': -1,
                '': 0,
                'da': 1,
                'h': 2,
                'k': 3,
                'M': 6,
                'G': 9,
                'T': 12,
                'P': 15,
                'E': 18,
                'Z': 21,
                'Y': 24,
            }

            exponent = prefixes[unit[0]]
            unit = unit[1:]
        else: 
            exponent = 0

        self.min_value = (min_value * 10 ** exponent) 
        self.max_value = (max_value * 10 ** exponent) 

        self.unit = unit
        self.value = (value * 10 ** exponent)

class Gate(Quantity):
    def __init__(self, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit)
        self.device = device

    def update_value(self, value: str, unit: str):
        super().__init__(value, self.min_value, self.max_value, unit)
        self.set_value()

    def increment_value(self, dX: float, unit: str):
        dX = Quantity(dX, 0, 0, unit)
        # clamp between bounds
        new_value = max(min(self.value + dX.value, self.max_value), self.min_value)

        super().__init__(new_value, self.min_value, self.max_value, self.unit)
        self.set_value()

    def set_value(self):
        # set current value to device
        pass 

    def measure_value(self): 
        # measure current value 
        pass

class Plunger(Gate):
    def __init__(self, name: str, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit, device)
        self.name = name

class Barrier(Gate):
    def __init__(self, name: str, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit, device)
        self.name = name

class Reservoir(Gate):
    def __init__(self, name: str, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit, device)
        self.name = name

class Ohmic(Gate):
    def __init__(self, name: str, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit, device)
        self.name = name
    
class Screening(Gate):
    def __init__(self, name: str, value: float, min_value: float, max_value: float, unit: str, device: str) -> None:
        super().__init__(value, min_value, max_value, unit, device)
        self.name = name

class DUT:
    def __init__(self, DUT_info: dict) -> None:
        pass

In [59]:
# TEST GATES
O2 = Ohmic("O2", 0.0, -1.0, 1.0, "V", "")
for i in range(50):
    O2.increment_value(-2 * i, "mV")
    print(O2.value, O2.unit)

0.0 V
-0.002 V
-0.006 V
-0.012 V
-0.02 V
-0.03 V
-0.041999999999999996 V
-0.055999999999999994 V
-0.072 V
-0.09 V
-0.11 V
-0.132 V
-0.156 V
-0.182 V
-0.21 V
-0.24 V
-0.272 V
-0.30600000000000005 V
-0.3420000000000001 V
-0.38000000000000006 V
-0.42000000000000004 V
-0.462 V
-0.506 V
-0.552 V
-0.6000000000000001 V
-0.6500000000000001 V
-0.7020000000000002 V
-0.7560000000000002 V
-0.8120000000000003 V
-0.8700000000000003 V
-0.9300000000000004 V
-0.9920000000000004 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V
-1.0 V


# Global Turn On

In [ ]:
def measure_global_turn_on(DUT: object, V_bias: object, dV: object, min_current_threshold: float, max_current_threshold: float) -> float:
    # DUT -> Design under Test
    DUT.O1.update_value(value=V_bias.value, unit=V_bias.unit)
    DUT.O2.update_value(value=V_bias.value, unit=V_bias.unit)
    
    # Keep incrementing all the gate voltages until minimum current threshold is reached
    while DUT.O3.value < min_current_threshold and DUT.O4.value < min_current_threshold:
        DUT.O3.measure()
        DUT.O4.measure()
        for gate in DUT.gates():
            gate.increment_value(dV.value, unit=dV.unit)
    V_min = np.average([gate.value for gate in DUT.gates()])

    # Keep incrementing all the gate voltages until maximum current threshold is reached
    while DUT.O3.value < max_current_threshold and DUT.O4.value < max_current_threshold:
        DUT.O3.measure()
        DUT.O4.measure()
        for gate in DUT.gates():
            gate.increment_value(dV.value, unit=dV.unit)
            
    V_max = np.average([gate.value for gate in DUT.gates()])

    V_error = Quantity(0, 0, 0, "mV")
    global_turn_on_dist = Quantity(np.abs(V_max - V_min),min_value=np.abs(V_max - V_min) - V_error.value, max_value=np.abs(V_max - V_min) + V_error.value, unit="V")
    
    return global_turn_on_dist

# Center Screening Gate Pinch-Off

In [ ]:
def f(x, a, b, x0, y0):
    return a * (np.arccosh(b(x-x0)))**2 + y0

def pinch_off_center_screening_gate(DUT: object, dV: object, global_turn_on_distance: object):

    # Use global turn on distance to determine voltage sweep ranges
    num_of_increments = (3.5 * global_turn_on_distance.value) / dV.value

    S2_values = [DUT.S2.value]
    O2_values = [DUT.O2.measure()]
    O4_values = [DUT.O4.measure()]

    for i in range(num_of_increments):
        # Decrement S2 and record S2 voltage and O2/4 currents
        DUT.S2.increment_value(-dV.value, unit=dV.unit)

        S2_values.append(DUT.S2.value)
        O2_values.append(DUT.O2.measure())
        O4_values.append(DUT.O4.measure())

    # Obtain dIdV curves to fit and extract width of pinch-off curves
    dO2dS2 = np.gradient(O2_values, S2_values)
    dO4dS2 = np.gradient(O4_values, S2_values)

    # Find voltage when sum of squares is a maximum
    sum_of_squares = np.array(dO2dS2)**2 + np.array(dO4dS2)
    max_value_index = np.argmax(sum_of_squares)
    max_voltage = S2_values[max_value_index] 

    O2_params, O2_covariance = curve_fit(f, S2_values, dO2dS2)
    O4_params, O4_covariance = curve_fit(f, S2_values, dO4dS2)

    _, O2_width, _, _ = O2_params
    _, O4_width, _, _ = O4_params

    # Set S2 two widths under critical pinch-off voltage
    S2_pinch_off_val = max_voltage - (2 * np.avg([O2_width, O4_width]))
    V_error = Quantity(0, 0, 0, "mV")    
    DUT.S2.update_value(S2_pinch_off_val, S2_pinch_off_val-V_error, S2_pinch_off_val+V_error, "V")

# Bias Compensation

In [ ]:
def f(x, m, b):
    return m * x + b

def measure_ohmic_bias(DUT: object, ohmic_gate_bias: object, ohmic_gate_current: object, dV_bias: object, dV_gates: object):

    # Take 50% of the true min max range to sweep over
    percent = 0.5
    range = np.abs(ohmic_gate_bias.max_value - ohmic_gate_bias.min_value)
    num_of_increments = 0.5 * (range / dV_bias.value)

    ohmic_gate_values = np.linspace([ohmic_gate_bias.min_value, ohmic_gate_bias.max_value] - (percent / 2)*range, [ohmic_gate_bias.min_value, ohmic_gate_bias.max_value] + (percent / 2)*range, num_of_increments)

    # Set ohmic gate bias to starting point, below the max
    ohmic_gate_bias.update_value(
        np.avg([ohmic_gate_bias.min_value, ohmic_gate_bias.max_value]) + (percent / 2) * range, unit="V"
    )

    # Record current
    current_1_values = [ohmic_gate_current.measure()]

    # Sweep and record current - should be linear relationship
    for i in range(num_of_increments):
        ohmic_gate_bias.increment_value(-dV_bias.value, unit=dV_bias.unit)
        current_1_values.append(ohmic_gate_current.measure())

    # Lower all gate voltages and repeat
    for gate in DUT.gates():
            gate.increment_value(dV_gates.value, unit=dV_gates.unit)

    # Set ohmic gate bias to starting point, below the max
    ohmic_gate_bias.update_value(
        np.avg([ohmic_gate_bias.min_value, ohmic_gate_bias.max_value]) + (percent / 2) * range, unit="V"
    )

    # Record current
    current_2_values = [ohmic_gate_current.measure()]

    # Sweep and record current - should be linear relationship
    for i in range(num_of_increments):
        ohmic_gate_bias.increment_value(-dV_bias.value, unit=dV_bias.unit)
        current_2_values.append(ohmic_gate_current.measure())

    # Fit to linear functions, extract POI
    params_1, _ = curve_fit(f, ohmic_gate_values, current_1_values)
    params_2, _ = curve_fit(f, ohmic_gate_values, current_2_values)

    m1, b1 = params_1
    m2, b2 = params_2

    ohmic_gate_bias = (b2-b1) / (m1-m2)
    ohmic_current_bias = f(ohmic_gate_bias, m1, b1)

    return ohmic_gate_bias, ohmic_current_bias

def measure_channel_resistance(ohmic_gate_bias: object, ohmic_gate_current: object, dV_bias: object):
    # Take 50% of the true min max range to sweep over
    percent = 0.5
    range = np.abs(ohmic_gate_bias.max_value - ohmic_gate_bias.min_value)
    num_of_increments = 0.5 * (range / dV_bias.value)

    ohmic_gate_values = np.linspace([ohmic_gate_bias.min_value, ohmic_gate_bias.max_value] - (percent / 2)*range, [ohmic_gate_bias.min_value, ohmic_gate_bias.max_value] + (percent / 2)*range, num_of_increments)

    # Set ohmic gate bias to starting point, below the max
    ohmic_gate_bias.update_value(
        np.avg([ohmic_gate_bias.min_value, ohmic_gate_bias.max_value]) + (percent / 2) * range, unit="V"
    )

    # Record current
    current_values = [ohmic_gate_current.measure()]

    # Sweep and record current - should be linear relationship
    for i in range(num_of_increments):
        ohmic_gate_bias.increment_value(-dV_bias.value, unit=dV_bias.unit)
        current_values.append(ohmic_gate_current.measure())

    # Fit to linear functions, extract POI
    params_1, _ = curve_fit(f, ohmic_gate_values, current_values)
    R, I0 = params_1 # linear IV curve

    return R

device = Device()
min_resistance_threshold = 1e-3
dV_bias = Quantity(0.1, 0, 0, "V")
dV_gates = Quantity(0.1, 0, 0, "V")
dV_screening = Quantity(0.1, 0, 0, "V")

R = measure_channel_resistance(device.O1, device.O4, dV_bias)
ohmic_gate_bias, ohmic_current_bias = measure_ohmic_bias(device, device.O1, device.O2, dV_bias, dV_gates)

while R > min_resistance_threshold:
    # Adjust screening gate
    device.S2.increment_value(-dV_screening.value, unit=dV_screening.unit)

    # abort if S2 value reaches minimum.
    if device.S2.value == device.S2.min_value:
        raise ValueError("Saturated S2, exiting loop.")
        break

    # Re-measure
    R = measure_channel_resistance(device.O1, device.O4, dV_bias)
    ohmic_gate_bias, ohmic_current_bias = measure_ohmic_bias(device, device.O1, device.O2, dV_bias, dV_gates)

# Individual Channel Turn-On

In [ ]:
def individual_channel_turn_on(channel: object, V_bias: object, dV: object, min_current_threshold: float, max_current_threshold: float):

    channel.ohmic_gate.update_value(V_bias.value, V_bias.unit)

     # Keep incrementing all the gate voltages until minimum current threshold is reached
    while channel.ohmic_current_gate.value < min_current_threshold:
        channel.ohmic_current_gate.measure()
        for gate in channel.gates():
            gate.increment_value(dV.value, unit=dV.unit)
    V_min = np.average([gate.value for gate in channel.gates()])

    # Keep incrementing all the gate voltages until maximum current threshold is reached
    while channel.ohmic_current_gate.value < max_current_threshold:
        channel.ohmic_current_gate.measure()
        for gate in channel.gates():
            gate.increment_value(dV.value, unit=dV.unit)
    V_max = np.average([gate.value for gate in channel.gates()])

    return V_min, V_max

Re-measure the screening gate pinch-off value, just in-case step above has created a change in behaviour.

# Individual Reservoir Turn-On / Pinch-Off